Name: Ahmet Çağrı Düzgün

I hereby declare that I observed the honour code of the university when preparing the homework.

## Solution to 16.3-5 and 18

#### 16.3

Clearly, a variable depends on the variable which comes just before it.A variable is mostly the same as the one coming just before it. Based on my observations, a change can occur in two ways. First, Trend can be changed which we can reflect with a change in $S_{t}=b$. Or a noise can occur without changing the trend when $x_{t}=b$. Hence, I can suggest the following model
<img src="files/Q1.PNG">

#### 16.4
Based on the observation from the data, I can suggest the following model:
<img src="files/Q2.PNG">

#### 16.5 
Even though a complicated model could be better, the following model also works:

<img src="files/Q3.PNG">

#### 18.1  
<img src="files/D.PNG">

#### 18.2  
<img src="files/G.PNG">
<img src="files/F.PNG">

#### 18.3 
We can compute each term one by one and sum them up:
$N^2(N-1) + N + +N(N-1) + N^2(N-1) + N = 2(N+1)^2(N-1)$

#### 18.4 

a)The statement is true. That is because there are two paths we can go from A to M. In A-T-F-L-M, F blocks the path. In A-T-F-D-B-M, D blocks the path

b)The statement is false. Given X, we have knowledge of F. Hence, A-T-F-L-M is open now. 

c)The statement is false. Given X, we have knowledge of F as it is in the previous question. 

d) The statement is true. Not given F, the statement is false. But, given F, The path from X to F is unlocked. Hence, they are conditionally independent.

e)The statement is false. There is a path from L to X( L-F-X). Given D, no effect occurs on this path. 


## Programming Homework 3

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [3]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


In [4]:
import numpy as np
T = np.array(T,dtype='float')
import itertools

def sample(N):
    text = ''
    letter = '.'
    letter_id = letter2idx[letter]
    text += letter
    for i in range(1,N):  
        row = T[letter_id,:]
        row = row / np.sum(row)
        letter_id = np.random.choice(len(alphabet),1,p=row)[0]
        letter = alphabet[letter_id]
        text += letter
    return text

def prob(text):
    p = 1
    for i in range(1,len(text)):
        curr = letter2idx[text[i]]
        prev = letter2idx[text[i-1]]
        p *= T[curr,prev]
    return p

def index_to_text(index,rep):
    text = []
    for i in range(rep):
        text.insert(0, alphabet[index % len(alphabet)])
        index /= len(alphabet)
        index = int(index)
    return text
    

def generate(text):
    count = 0
    indices = np.where(text == '_')[0]
    ps = []
    max_p = 0
    for letter_ids in itertools.product(range(len(alphabet)), repeat=len(indices)):
        for i,id in enumerate(letter_ids):
            text[indices[i]] = alphabet[id]
        p = prob(text)
        ps.append(p)
        
        count += 1
        if count > 20000000:
            print('FAIL')
            break 
    ps = np.array(ps)
    ps = ps / np.sum(ps)
    max_index = np.argmax(ps)
    rand_index = np.random.choice(len(alphabet)**len(indices),1,p=ps)[0]
    return index_to_text(max_index,len(indices)) , index_to_text(rand_index,len(indices)), ps[max_index], ps[rand_index]
        

def fill_missings(text,random=False):
    text = '.'+text
    text = np.array(list(text))
    indices = np.where(text == '_')[0]
    start, end = 0, 0
    p = 1
    for i in range(1,len(text)):
        if text[i] == '_':
            continue
        else:
            end = i
            if end >= len(text):
                break
            if end-start > 1:
                max_text, rand_text, pm, pr = generate(text[start:end+1])
                if random:
                    text[start+1:end] = rand_text
                    p *= pr
                else:
                    text[start+1:end] = max_text
                    p *= pm
            start = i
    end = len(text)
    if end - start > 1:
        max_text, rand_text, pm, pr = generate(text[start:end])
        if random:
            text[start+1:end] = rand_text
            p *= pr
        else:
            text[start+1:end] = max_text
            p *= pm
    return ''.join(text)[1:], np.log(p)

print('1) Random Strings:')
for i in [5,10,20,50,100]:
    print('N='+str(i)+':\t',sample(i))
print('\n')
print('2) Filling with random letter(s):')
for string in test_strings:
    print(string)
    new_string, logp = fill_missings(string,True)
    print(new_string)
    print('logP: '+str(logp))
    print('===')

print('\n')
print('3) Filling with MAP letter(s):')
for string in test_strings:
    print(string)
    new_string, logp = fill_missings(string,False)
    print(new_string)
    print('logP: '+str(logp))
    print('===')


1) Random Strings:
N=5:	 .ther
N=10:	 .m.am.e.s.
N=20:	 .rye.u.strsey.fr.wh.
N=50:	 .t.ir.the.ininon.on.ipeerarof.here.o.wid.ou.s.he.w
N=100:	 .ch.d.balatrnde.icikekeeree.n.t.plumete.trst.metufinyeveain.in.f.icce.cies.d.ld.at.t.g.teavere.had.a


2) Filling with random letter(s):
th__br__n.f_x.
thgebr.dn.fix.
logP: -9.22651443544
===
_u_st__n_.to_be._nsw_r__
ru.st.dna.toybe.dnsw.rac
logP: -13.4671867078
===
i__at_._a_h_n_._e_r_i_g
i.natr.taeh.na.yetroi.g
logP: -20.3391545376
===
q___t.___z._____t.__.___.__.
q.sot.traz..rarot.ya.te..eh.
logP: -41.6614947572
===


3) Filling with MAP letter(s):
th__br__n.f_x.
th.ebra.n.f.x.
logP: -5.35716554182
===
_u_st__n_.to_be._nsw_r__
suost.dna.toybe.dnsw.ruq
logP: -10.1342152768
===
i__at_._a_h_n_._e_r_i_g
ihtata.naehtna.deerei.g
logP: -13.4552636401
===
q___t.___z._____t.__.___.__.
q.eht.dnaz.eh.eht.eh.eht.eh.
logP: -19.9078091241
===
